# Full version coming soon...
---

In [8]:
import sys
import os
########################################################################
from deepdreaming import img, display, deepdream as dd
from deepdreaming.utils import read_image_net_classes
from deepdreaming.config import DreamConfig, GradSmoothingMode

import numpy as np
from matplotlib import pyplot as plt

import torch
from torchvision.models import resnet50, vgg16, inception_v3
from torchvision.models import ResNet50_Weights, VGG16_Weights, Inception_V3_Weights
import cv2 as cv
########################################################################
SEED = 42
def set_seeds(seed_value):
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    torch.cuda.manual_seed_all(seed_value)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seeds(SEED)
########################################################################
%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
TARGET_SHAPE = (224, 224, 3)
IMAGE_NET_CLASSES_PATH = "input/image_net_classes/imagenet_classes.txt"
INPUT_IMAGES_DIR = "input/deepdream-sample-images"
OUTPUT_IMAGES_DIR = "output"

assert os.path.exists(IMAGE_NET_CLASSES_PATH), "Please provide a valid `IMAGE_NET_CLASSES_PATH`"
assert os.path.exists(INPUT_IMAGES_DIR), "Please provide a valid `INPUT_IMAGES_DIR`"
assert os.path.exists(OUTPUT_IMAGES_DIR), "Please provide a valid `DEEPDREAM_OUTPUT_PATH`"



#########################################
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
MODEL_TYPE = inception_v3
WEIGHTS = Inception_V3_Weights.DEFAULT
#########################################

MODEL = MODEL_TYPE(weights=WEIGHTS).to(DEVICE).eval()
MODEL.requires_grad_(False)
IMAGE_NET_CLASSES = read_image_net_classes(IMAGE_NET_CLASSES_PATH)

print(MODEL)

Inception3(
  (Conv2d_1a_3x3): BasicConv2d(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2a_3x3): BasicConv2d(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2b_3x3): BasicConv2d(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (Conv2d_3b_1x1): BasicConv2d(
    (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_4a_3x3): BasicConv2d(
    (conv): Conv2d(80, 192, kernel_size=(3, 3), stri

In [10]:
# deepdream = dd.DeepDream(MODEL, ["layer3[5].relu"])
# deepdream = dd.DeepDream(MODEL, ["features[13]", "features[22]", "features[25]", "features[27]"])
deepdream = dd.DeepDream(MODEL, ["Conv2d_1a_3x3.conv"])
sample_images = [os.path.join(INPUT_IMAGES_DIR, file_name) for file_name in os.listdir(INPUT_IMAGES_DIR)]
sample_config = DreamConfig(
    pyramid_layers=5,
    pyramid_ratio=3 / 4,
    shift_size=128,
    num_iter=10,
    learning_rate=0.08,
    grad_smoothing=GradSmoothingMode.GaussianSmoothing,
    grad_smoothing_kernel_size=5,
    grad_smoothing_gaussian_sigmas=(0.5, 1, 1.5),
    grad_smoothing_gaussian_blending_weights=(10, 2, 3),
)

# nice configs:
# num_iter = 10
#   -> layers=9, ratio=4/5
#   -> layers=3, ratio=0.42

sample_images[0] = os.path.join(INPUT_IMAGES_DIR, "guys.jpg")
for sample_image in sample_images[::3]:
    # read 'n' dream
    image: np.ndarray = img.io.read_image(sample_image, TARGET_SHAPE)
    out = deepdream.dream(image, config=sample_config)

    # compare 'n' show
    display.display_two_img(image, out, figsize=(8, 8))
    plt.show()

    # transform for OpenCV
    out = img.proc.to_cv(out)

    # save
    file = f"{sample_image.split('/')[-1]}"
    fullpath = os.path.join(OUTPUT_IMAGES_DIR, file)
    success = cv.imwrite(fullpath, out)
    assert success, f"Image {fullpath} is not saved successfully..."

RuntimeError: Calculated padded input size per channel: (2 x 2). Kernel size: (3 x 3). Kernel size can't be greater than actual input size

## Using reference image
**Dreaming with the reference image provided**

In [ ]:
get_path = lambda prefix, name: os.path.join(prefix, name)

# some experimental layer with cool results - feel free to change!
deepdream = dd.DeepDream(MODEL, ["features[22]"])

# config for guided deepdream
guided_dream_config = DreamConfig(gradient_norm=True, pyramid_layers=4, num_iter=10)

### Dart Vader + Yoda

In [ ]:
image = img.io.read_image(get_path(INPUT_IMAGES_DIR, "yoda.jpeg"), TARGET_SHAPE)
reference = img.io.read_image(get_path(INPUT_IMAGES_DIR, "space.jpeg"), TARGET_SHAPE)

out = deepdream.dream(image, reference, config=guided_dream_config)
display.display_two_img(image, out, figsize=(8, 8))

In [ ]:
from deepdreaming import smoothing

fig, axis = plt.subplots(1, 2, figsize=(12, 12))
ax0, ax1 = axis.flatten()

image = img.io.read_image("/home/vlad/source/deepdreaming/input/deepdream-sample-images/noisy_image.jpg")
ax0.imshow(image)
image = img.proc.to_tensor(image)

out = smoothing.gaussian_smoothing(image, kernel_size=9, sigmas=(1, 2, 3))
out = img.proc.to_image(out)
ax1.imshow(out)